In [1]:
import numpy as np
from Classes.Analysis import Analysis
from Classes.GraphMaker import GraphMaker

class System(Analysis):
    def __init__(self, graph_location):
        super().__init__(graph_location)

    def new_system2(self, t, Y, *params):
        e_matrix, v_matrix, K_matrix, r_matrix, tau_matrix, B_matrix, M, dimension = params
        def g(N, v, K):
            print(K_matrix.size)
            return (N @ v) / (N + K)
        N = Y[:dimension]
        U = Y[dimension:dimension*2]
        I = Y[dimension*2:dimension*2 + dimension*M].reshape(dimension, M)
        P = Y[dimension*2 + dimension*M:]
        new_N = (e_matrix @ g(N, v_matrix, K_matrix)) @ (U + np.sum(I, axis=1))
        new_U = (g(N, v_matrix, K_matrix) @ U) - ((r_matrix @ U) * P)
        M_tau = np.nan_to_num(M / tau_matrix, nan=0, posinf=0, neginf=0)
        new_I = np.zeros_like(I)
        for column in range(M):
            if column == 0:
                new_I[:, column] = ((r_matrix @ U) @ P) - (M_tau @ I[:, column])
            else:
                new_I[:, column] = M_tau[:, column] @ (I[:, column] - I[:, column-1])
        new_P = ((B_matrix @ M_tau) @ I[:, -1]) - ((r_matrix @ (U + np.sum(I, axis=1))) @ P)
        flattened_y1 = self.flatten_lists_and_matrices(new_N, new_U, new_I, new_P)
        return self.check_cutoff(flattened_y1, dimension)

graph = GraphMaker()
graph = graph.graph

graph = System('example.gexf')
phage_nodes = graph.get_nodes_of_type('P')
bacteria_nodes = graph.get_nodes_of_type('B')
resource_nodes = graph.get_nodes_of_type('R')
environemnt_nodes = graph.get_nodes_of_type('E')

R0 = graph.initialize_initial_concentration(resource_nodes)
U0 = graph.initialize_initial_population(bacteria_nodes)
I0 = graph.initialize_new_matrix(graph.max_length, graph.M)
P0 = graph.initialize_initial_population(phage_nodes)
e_matrix = graph.initialize_new_parameter_from_edges(bacteria_nodes, resource_nodes, 'e')
v_matrix = graph.initialize_new_parameter_from_edges(bacteria_nodes, resource_nodes, 'v')
K_matrix = graph.initialize_new_parameter_from_edges(bacteria_nodes, resource_nodes, 'K')
r_matrix = graph.initialize_new_parameter_from_edges(bacteria_nodes, phage_nodes, 'r')
tau_matrix = graph.initialize_new_parameter_from_edges(bacteria_nodes, phage_nodes, 'tau')
B_matrix = graph.initialize_new_parameter_from_edges(bacteria_nodes, phage_nodes, 'Burst Size')

y0_flattened = graph.flatten_lists_and_matrices(R0, U0, I0, P0)

solved_system = graph.solve_system(graph.new_system2, y0_flattened, e_matrix, v_matrix, K_matrix, r_matrix, tau_matrix, B_matrix, int(graph.M), graph.max_length)

solved_y = solved_system.y
new_N = solved_y[:graph.max_length]
new_U = solved_y[graph.max_length:graph.max_length*2]
new_I = solved_y[graph.max_length*2:graph.max_length*2 + graph.max_length*int(graph.M)]
new_P = solved_y[graph.max_length*2 + graph.max_length*int(graph.M):]

2025-02-03 20:04:55.669 Python[97244:1443907] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-03 20:04:55.669 Python[97244:1443907] +[IMKInputSession subclass]: chose IMKInputSession_Modern


: 